In [6]:
import pandas as pd
import os
import shutil
from tqdm.notebook import tqdm
import subprocess

class SMT_test:
    chart = None
    def __init__(self, benchmarks_path='../', chart_path='benchmarks', result_path='results'):
        self.benchmarks_path = benchmarks_path
        self.chart_path = chart_path
        self.result_path = result_path

    def get_unknown_path(self, solver, log_path, num_groups, prefix):
        full_file_path = f'{self.chart_path}/{log_path}_unknown.log'
        self.file = open(full_file_path, "w")
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        count = 0
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    if solver == 'z3':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == 'run':
                                result = 'timeout'
                            if result[:3] == 'tac':
                                result = 'timeout'
                                f.readline()
                            info = f.readline()[:-1]
                            if result == 'unknown':
                                self.file.write(f"{prefix}{info}\n")
                                count += 1
                    elif solver == 'cvc5':
                        while f.readline():
                            info = f.readline().rstrip()
                            result = info
                            if info == 'unknown':
                                info = f.readline().rstrip()
                                if len(info) < 10:
                                    if len(info) > 1:
                                        result = info
                                    info = f.readline().rstrip()
                            elif len(info) > 10:
                                result = "unknown"
                            # elif len(info) == 0:
                            #     break
                            else:
                                info = f.readline().rstrip()
                            if result == 'unknown':
                                self.file.write(f"{prefix}{info.split(' : ')[0]}\n")
                                # print(info)
                                count += 1
                    elif solver == 'yices2':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == 'QF_':
                                info = result
                                result = 'timeout'
                            else:
                                info = f.readline()[:-1]
                            if result == 'unknown':
                                self.file.write(f"{prefix}{info}\n")
                                count += 1
                    elif solver == 'mathsat5':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == './s':
                                result = 'timeout'
                            if result[:3] == 'tcm':
                                result = 'timeout'
                                info = f.readline()
                                while info[:3] == 'tcm':
                                    info = f.readline()
                            if result[:3] == '(er':
                                result = 'not support'
                            info = f.readline()[:-1]
                            if result == 'unknown':
                                self.file.write(f"{prefix}{info}\n")
                                count += 1
                    elif solver == 'smtrat':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == './s':
                                result = 'timeout'
                            if result[:3] == 'QF_':
                                info = result
                                result = 'none'
                            else:
                                info = f.readline()[:-1]
                            if result == 'unknown':
                                self.file.write(f"{prefix}{info}\n")
                                count += 1
                except:
                    print(file)
        print(f"count: {count}")
        self.file.close()
        
        output_directory = f"{self.chart_path}/{log_path}_unknown"
        if not os.path.exists(output_directory):
            os.mkdir(output_directory)
        else:
            shutil.rmtree(output_directory)
            os.mkdir(output_directory)
        shell_command = f'split -l $((($(wc -l < {full_file_path}) + {num_groups} - 1) / {num_groups})) {full_file_path} "{output_directory}/group_"'
        try:
            subprocess.run(shell_command, shell=True, check=True)
            print("Shell命令执行成功")
        except subprocess.CalledProcessError as e:
            print(f"Shell命令执行失败: {e}")

In [7]:
str = SMT_test()
# str.init_chart('QF_S', 'base.xlsx', 'QF_S.xlsx')
# str.init_chart('QF_SLIA', 'base.xlsx', 'QF_SLIA.xlsx')
str.get_unknown_path('cvc5', 'QF_S', 50, "/home/hanrui/")
str.get_unknown_path('cvc5', 'QF_SLIA', 50, "/home/hanrui/")

results/cvc5/QF_S:   0%|          | 0/100 [00:00<?, ?it/s]

count: 209
Shell命令执行成功


results/cvc5/QF_SLIA:   0%|          | 0/100 [00:00<?, ?it/s]

count: 1383
Shell命令执行成功
